In [ ]:
elastic_rods_dir = '../../../elastic_rods/python/'
weaving_dir = '../../'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj,
                                   write_linkage_ribbon_output_florin)

from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods


import vis.fields
import matplotlib.cm as cm
import time

In [ ]:
# Set to True in the options below if you already optimized a specific linkage
# and you would like to reuse the optimized linkage. Loading is performed in
# place of the full design optimization.
# NOTE: Doesn't seem to work at the moment, leave at False
LOAD_OPTIMIZED_DOFS = False

# Set to False if strips cannot be labeled into families for a model
USE_FAMILY_LABEL = False

In [ ]:
# # Heart Coarse 1
# default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
# RIBBON_CS = [1/150, 1/15]
# MODEL_NAME = "heart_coarse_1"
# MODEL_PATH = osp.join(weaving_dir + 'scaled_objs/models/{}.obj'.format(MODEL_NAME))
# SUBDIVISION_RESOLUTION = 20
# SMOOTHING_WEIGHT = 10
# REGULARIZATION_WEIGHT = 1
# INPUT_SURFACE_PATH = osp.join(weaving_dir + 'scaled_objs/surface_models/{}.obj'.format(MODEL_NAME))
# RIBBON_NAME = "heart_coarse_1_strip"

In [ ]:
# Regular Torus 1
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
RIBBON_CS = [1/200, 1/20]
MODEL_NAME = "regular_torus_1"
MODEL_PATH = osp.join(weaving_dir + 'normalized_objs/models/{}.obj'.format(MODEL_NAME))
SUBDIVISION_RESOLUTION = 20
SMOOTHING_WEIGHT = 10
REGULARIZATION_WEIGHT = 0
INPUT_SURFACE_PATH = osp.join(weaving_dir + 'normalized_objs/surface_models/{}.obj'.format(MODEL_NAME))
RIBBON_NAME = "{}_strip".format(MODEL_NAME)

In [ ]:
def initialize_linkage(surface_path = INPUT_SURFACE_PATH, useCenterline = True, cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION, model_path = MODEL_PATH):
    l = elastic_rods.SurfaceAttractedLinkage(surface_path, useCenterline, model_path, subdivision_res, False, InterleavingType.triaxialWeave)
    l.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))
    l.set_holdClosestPointsFixed(True);
    l.set_attraction_tgt_joint_weight(0.01);
    l.attraction_weight = 100;
    return l

In [ ]:
def get_linkage_eqm(l, opt, cam_param = default_camera_parameters, target_surf = None):
    elastic_rods.compute_equilibrium(l, options = opt)
    if (target_surf is None):
        view = linkage_vis.LinkageViewer(l, width=1024, height=640)
    else:
        view = linkage_vis.LinkageViewerWithSurface(l, target_surf, width=1024, height=640)
    view.setCameraParams(cam_param)
    return l, view

In [ ]:
OPTS = elastic_rods.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 10;
OPTS.beta = 1e-8
OPTS.niter = 100
OPTS.verboseNonPosDef = False

In [ ]:
rw = 0.001
sw = 0.001

# Straight Ribbon Optimization

In [ ]:
with so(): straight_linkage = initialize_linkage(surface_path = INPUT_SURFACE_PATH, useCenterline = True, model_path = MODEL_PATH, cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION)
straight_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = False)
straight_save_tgt_joint_pos = straight_linkage.jointPositions();
straight_linkage_view = linkage_vis.LinkageViewer(straight_linkage)
straight_linkage_view.show()

### Figure 3C bottom

In [ ]:
# Need a function to write out the save_tgt_joint_pos

In [ ]:
def straight_callback(prob, i):
    if (i % 20) != 0: return
    straight_linkage_view.update()

In [ ]:
straight_dpo = elastic_rods.get_designParameter_optimizer(straight_linkage, rw, sw, callback=straight_callback)
straight_dpo.options.niter = 200
straight_dpp = straight_dpo.get_problem()

In [ ]:
with so(): straight_cr = straight_dpo.optimize()

In [ ]:
with so(): elastic_rods.compute_equilibrium(straight_linkage, options = OPTS)
straight_linkage_view.update()

In [ ]:
OPTS.niter = 200
useCenterline = True
straight_optimizer = elastic_rods.WeavingOptimization(straight_linkage, INPUT_SURFACE_PATH, useCenterline, equilibrium_options=OPTS, pinJoint = 0, useFixedJoint = False)
straight_optimizer.set_target_joint_position(straight_save_tgt_joint_pos)
straight_linkage_view.update()

In [ ]:
straight_optimizer.rl_regularization_weight = 0
straight_optimizer.smoothing_weight = 0
straight_optimizer.beta = 500000.0
straight_optimizer.gamma = 1
algorithm = elastic_rods.WeavingOptAlgorithm.NEWTON_CG
def straight_update_viewer():
    straight_linkage_view.update()

In [ ]:
straight_optimizer.WeavingOptimize(algorithm, 2000, 1.0, 1e-2, straight_update_viewer)

In [ ]:
# straight_optimizer.setLinkageAttractionWeight(1)
# straight_optimizer.set_holdClosestPointsFixed(False)

In [ ]:
# straight_optimizer.WeavingOptimize(algorithm, 2000, 1.0, 1e-2, straight_update_viewer)

In [ ]:
straight_optimizer_energy = straight_linkage.energy()

In [ ]:
validation_straight_linkage = straight_optimizer.getLinesearchWeaverLinkage()

In [ ]:
validation_straight_linkage.attraction_weight = 1e-7
with so(): elastic_rods.compute_equilibrium(validation_straight_linkage, options = OPTS)
validation_straight_view = linkage_vis.LinkageViewer(validation_straight_linkage, width=1024, height=640)

In [ ]:
validation_straight_energy = validation_straight_linkage.energy()

In [ ]:
print(abs((validation_straight_energy-straight_optimizer_energy)/straight_optimizer_energy))

In [ ]:
validation_straight_view.show()

### Figure 3B Top

In [ ]:
straight_ribbons = order_segments_by_ribbons(validation_straight_linkage)
write_linkage_ribbon_output_florin(validation_straight_linkage, straight_ribbons, SUBDIVISION_RESOLUTION, 'nature_figure3_straight_linkage', True)

### Figure 3B Bottom

In [ ]:
straight_distance_to_surface = np.array(validation_straight_linkage.get_squared_distance_to_target_surface((validation_straight_linkage.visualizationGeometry()[0]).flatten()))
straight_distance_to_surface = np.sqrt(straight_distance_to_surface)
ribbon_distance_scale = 0.66/np.max(straight_distance_to_surface)
print(ribbon_distance_scale)
straight_distance_to_surface *= 0.66/np.max(straight_distance_to_surface)  
export_linkage_geometry_to_obj(validation_straight_linkage, 'nature_figure3_straight_linkage.obj', straight_distance_to_surface)

In [ ]:
straight_distance_to_surface = np.array(validation_straight_linkage.get_squared_distance_to_target_surface((validation_straight_linkage.visualizationGeometry()[0]).flatten()))
straight_distance_to_surface = np.sqrt(straight_distance_to_surface)
print(np.max(straight_distance_to_surface))

# Curved Ribbon Optimization

In [ ]:
def get_average_distance_to_target_surface(linkage):
    distance_to_surface = np.array(linkage.get_squared_distance_to_target_surface((linkage.visualizationGeometry()[0]).flatten()))
    distance_to_surface = np.sqrt(distance_to_surface)
    return np.sum(distance_to_surface)/len(distance_to_surface)
def get_average_distance_to_target_joint(linkage):
    jointPosDiff = linkage.jointPositions() - curved_save_tgt_joint_pos
    distances = [la.norm(jointPosDiff[3*x:3*(x+1)]) for x in range(linkage.numJoints())]
    return np.sum(distances)/linkage.numJoints()

In [ ]:
with so(): initial_linkage = initialize_linkage(surface_path = INPUT_SURFACE_PATH, useCenterline = True, model_path = MODEL_PATH, cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION)
with so(): elastic_rods.compute_equilibrium(initial_linkage, options = OPTS)
initial_energy = initial_linkage.energy(energyType=elastic_rods.SurfaceAttractionEnergyType.Elastic)
initial_rest_length = initial_linkage.totalRestLength()

In [ ]:
print(initial_energy, initial_rest_length)

In [ ]:
with so(): curved_linkage = initialize_linkage(surface_path = INPUT_SURFACE_PATH, useCenterline = True, model_path = MODEL_PATH, cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION)
curved_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
curved_save_tgt_joint_pos = curved_linkage.jointPositions();
curved_linkage_view = linkage_vis.LinkageViewer(curved_linkage)
curved_linkage_view.show()

In [ ]:
iterateData = []
!mkdir -p nature_figure3_dps_iterates
!rm -f dps_iterates/*.msh dps_iterates/*.png
prev_vars = curved_linkage.getExtendedDoFsPSRL()
prev_time_stamp = 0
def curved_callback(prob, i):
    global prev_vars
    global prev_time_stamp
    curr_vars = curved_linkage.getExtendedDoFsPSRL()
    iterateData.append([prob.restKappaSmoothness(),
                        curved_linkage.totalRestLength(),
                        curved_linkage.designParameterSolve_energy(),
                        get_average_distance_to_target_surface(curved_linkage),
                        get_average_distance_to_target_joint(curved_linkage),
                        time.time() - prev_time_stamp,
                        np.linalg.norm(curr_vars - prev_vars), 
                        prob.weighted_energy(),
                        prob.weighted_smoothness(),
                        prob.weighted_length(),
                        curr_vars])
    prev_time_stamp = time.time()
    prev_vars[:] = curr_vars
    if (i % 20) != 0: return
    curved_linkage_view.update()

In [ ]:
# iterateData = []
# iterateSegmentRL = []
# iterateEdgeRL = []
# !mkdir -p dps_iterates
# !rm -f dps_iterates/*.msh dps_iterates/*.png
# prev_vars = curved_linkage.getExtendedDoFsPSRL()
# def curved_callback(prob, i):
#     global prev_vars
#     curr_vars = curved_linkage.getExtendedDoFsPSRL()
#     curved_linkage.saveVisualizationGeometry(f'dps_iterates/it_{i}.msh', averagedMaterialFrames=True, averagedCrossSections=True)
#     iterateSegmentRL.append(curved_linkage.getPerSegmentRestLength())
#     iterateEdgeRL.append(np.array([s.rod.restLengths() for s in curved_linkage.segments()]).ravel())
#     iterateData.append([prob.restKappaSmoothness(),
#                         curved_linkage.totalRestLength(),
#                         curved_linkage.designParameterSolve_energy(),
#                         np.linalg.norm(curr_vars - prev_vars)])
#     prev_vars[:] = curr_vars
#     if (i % 20) != 0: return
#     curved_linkage_view.update()

In [ ]:
curved_dpo = elastic_rods.get_designParameter_optimizer(curved_linkage, rw, sw, callback=curved_callback)
curved_dpo.options.niter = 200
curved_dpp = curved_dpo.get_problem()

In [ ]:
prev_time_stamp = time.time()
with so(): curved_cr = curved_dpo.optimize()

In [ ]:
total_time = 0
for data in iterateData:
    total_time += (data[5])
print(total_time)

In [ ]:
# Visualization of iterate geometry and statistics
import PIL
iterateData = np.array(iterateData)
def concat_v(im1, im2):
    dst = PIL.Image.new('RGB', (im1.width, im1.height + im2.height), color=(255, 255, 255))
    dst.paste(im1, (0, 0), im1)
    dst.paste(im2, (0, im1.height))
    return dst
def drawPlot(it):
    fig = plt.figure(figsize=(16, 8))
    gs = plt.GridSpec(2, 10, figure=fig)
    def plot(num, title):
        fig.add_subplot(gs[0, num])
        plt.title(title)
        plt.grid()
        plt.plot(iterateData[:, num])
        plt.scatter([it], [iterateData[it, num]])
    plot(0, 'smoothing term')
    plot(1, 'total length')
    plot(2, 'elastic energy')
    plot(3, 'dis to target surface')
    plot(4, 'dis to target joint')
    plot(5, 'step length')
    plot(7, 'weighted elastic')
    plot(8, 'weighted smooth')
    plot(9, 'weighted length')
    
def drawFrame(it):
    drawPlot(it)
    plt.savefig('plot.png', dpi=100)
    plt.close()
    !gmsh_offscreen -n dps_iterates/it_{it + 1}.msh misc/render.opt -o out.png > /dev/null 2> /dev/null
    return PIL.Image.open('plot.png')

In [ ]:
drawFrame(curved_cr.numIters())

In [ ]:
with so(): elastic_rods.compute_equilibrium(curved_linkage, options = OPTS)
curved_linkage_view.update()

In [ ]:
import linkage_optimization

In [ ]:
OPTS.niter = 200
useCenterline = True
curved_optimizer = linkage_optimization.WeavingOptimization(curved_linkage, INPUT_SURFACE_PATH, useCenterline, equilibrium_options=OPTS, pinJoint = 0, useFixedJoint = False)
curved_optimizer.set_target_joint_position(curved_save_tgt_joint_pos)
curved_linkage_view.update()

In [ ]:
curved_optimizer.rl_regularization_weight = 1
curved_optimizer.smoothing_weight = 10
curved_optimizer.beta = 500000.0
curved_optimizer.gamma = 1

In [ ]:
algorithm = linkage_optimization.WeavingOptAlgorithm.NEWTON_CG
opt_iterateData = []
prev_vars = curved_linkage.getExtendedDoFsPSRL()
prev_time_stamp = 0
def curved_update_viewer():
    global prev_vars
    global prev_time_stamp
    curr_vars = curved_linkage.getExtendedDoFsPSRL()
    opt_iterateData.append([curved_optimizer.restKappaSmoothness(), 
                            curved_linkage.totalRestLength(),
                            curved_linkage.energy(energyType=elastic_rods.SurfaceAttractionEnergyType.Elastic),
                            get_average_distance_to_target_surface(curved_linkage),
                            get_average_distance_to_target_joint(curved_linkage),
                            time.time() - prev_time_stamp,
                            np.linalg.norm(curr_vars - prev_vars), 
                            curved_optimizer.objective.terms[0].term.value(),
                            curved_optimizer.objective.terms[1].term.value(),
                            curved_optimizer.objective.terms[2].term.value(),
                            curved_optimizer.objective.terms[3].term.value(),
                            curr_vars])
    prev_time_stamp = time.time()
    prev_vars[:] = curr_vars
    curved_linkage_view.update()

In [ ]:
curved_optimizer.objective.terms[1].term.value

In [ ]:
prev_time_stamp = time.time()
curved_optimizer.WeavingOptimize(algorithm, 2000, 1.0, 1e-2, curved_update_viewer)

In [ ]:
curved_optimizer.setLinkageAttractionWeight(1e-5)
curved_optimizer.set_holdClosestPointsFixed(False)

In [ ]:
prev_time_stamp = time.time()
curved_optimizer.WeavingOptimize(algorithm, 2000, 1.0, 1e-2, curved_update_viewer)

In [ ]:
print(len(opt_iterateData))

In [ ]:
curved_optimizer_energy = curved_linkage.energy()
validation_curved_linkage = curved_optimizer.getLinesearchWeaverLinkage()
validation_curved_linkage.attraction_weight = 1e-7
with so(): elastic_rods.compute_equilibrium(validation_curved_linkage, options = OPTS)
validation_curved_view = linkage_vis.LinkageViewer(validation_curved_linkage, width=1024, height=640)
validation_curved_energy = validation_curved_linkage.energy()
print(abs((validation_curved_energy-curved_optimizer_energy)/curved_optimizer_energy))

In [ ]:
if not os.path.exists(RIBBON_NAME):
    os.makedirs(RIBBON_NAME)
get_all_curve_pattern(validation_curved_linkage, RIBBON_CS[1], SUBDIVISION_RESOLUTION, RIBBON_NAME, image_type='svg')

### Figure 3C Top

In [ ]:
curved_ribbons = order_segments_by_ribbons(curved_linkage)
write_linkage_ribbon_output_florin(curved_linkage, curved_ribbons, SUBDIVISION_RESOLUTION, 'nature_figure3_curved_linkage', True)

### Figure 3C Bottom

In [ ]:
curved_distance_to_surface = np.array(curved_linkage.get_squared_distance_to_target_surface((curved_linkage.visualizationGeometry()[0]).flatten()))
curved_distance_to_surface = np.sqrt(curved_distance_to_surface)
curved_distance_to_surface *= ribbon_distance_scale
export_linkage_geometry_to_obj(curved_linkage, 'nature_figure3_curved_linkage.obj', curved_distance_to_surface)

### Figure 3D Right

In [ ]:
# ribbon_folder_name = 'nature_figure3_ribbon'
# with so(): 
#     laser_cutting_linkage = initialize_linkage(surface_path = INPUT_SURFACE_PATH, useCenterline = True, model_path = MODEL_PATH, cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION)
#     for iter_idx in range(len(iterateData)):
#         laser_cutting_linkage.setExtendedDoFsPSRL(iterateData[iter_idx][-1])
#         if not os.path.exists(ribbon_folder_name):
#             os.makedirs(ribbon_folder_name)
#         get_all_curve_pattern(laser_cutting_linkage, RIBBON_CS[1], SUBDIVISION_RESOLUTION, ribbon_folder_name, image_type='svg', iteration_index = iter_idx, select_ribbon_index = [0])
        
#     for iter_idx in range(len(opt_iterateData)):
#         laser_cutting_linkage.setExtendedDoFsPSRL(opt_iterateData[iter_idx][-1])
#         if not os.path.exists(ribbon_folder_name):
#             os.makedirs(ribbon_folder_name)
#         get_all_curve_pattern(laser_cutting_linkage, RIBBON_CS[1], SUBDIVISION_RESOLUTION, ribbon_folder_name, image_type='svg', iteration_index = iter_idx + len(iterateData), select_ribbon_index = [0])

In [ ]:
# !ffmpeg -f image2 -framerate 10 -i nature_figure3_ribbon/0_%d.png -c:v libx264 -preset veryslow -qp 18 -pix_fmt yuv420p -y nature_figure3_ribbon/nature_figure3_ribbon.mp4 > /dev/null 2> /dev/null

### Figure 3D Left

In [ ]:
dps_energy = []
opt_energy = []
dps_smoothing = []
opt_smoothing = []
dps_rest_length = []
opt_rest_length = []
dps_total_absolute_curvature = []
opt_total_absolute_curvature = []

dps_distance_to_surface = []
opt_distance_to_surface = []
dps_distance_to_joint = []
opt_distance_to_joint = []

dps_time_stamps = []
opt_time_stamps = []
rk_offset = curved_linkage.numDoF()
rl_offset = rk_offset + (curved_linkage.numRestKappaVars() if curved_linkage.get_design_parameter_config().restKappa else 0)

with so(): visualization_linkage = initialize_linkage(surface_path = INPUT_SURFACE_PATH, useCenterline = True, model_path = MODEL_PATH, cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION)
visualization_linkage.attraction_weight = 1e-7

for iter_idx in range(len(iterateData)):
    dps_total_absolute_curvature.append(np.sum(np.abs(iterateData[iter_idx][-1][rk_offset:rl_offset])))
    dps_smoothing.append(iterateData[iter_idx][0])
    dps_rest_length.append(iterateData[iter_idx][1])
    dps_energy.append(iterateData[iter_idx][2])
    dps_time_stamps.append(iterateData[iter_idx][5])

    visualization_linkage.setExtendedDoFsPSRL(iterateData[iter_idx][-1])
    with so(): elastic_rods.compute_equilibrium(visualization_linkage, options = OPTS)
    dps_distance_to_surface.append(get_average_distance_to_target_surface(visualization_linkage))
    dps_distance_to_joint.append(get_average_distance_to_target_joint(visualization_linkage))


for iter_idx in range(len(opt_iterateData)):
    opt_total_absolute_curvature.append(np.sum(np.abs(opt_iterateData[iter_idx][-1][rk_offset:rl_offset])))
    opt_energy.append(opt_iterateData[iter_idx][2])
    opt_rest_length.append(opt_iterateData[iter_idx][1])
    opt_smoothing.append(opt_iterateData[iter_idx][0])
    opt_time_stamps.append(opt_iterateData[iter_idx][5])

    visualization_linkage.setExtendedDoFsPSRL(opt_iterateData[iter_idx][-1])
    with so(): elastic_rods.compute_equilibrium(visualization_linkage, options = OPTS)
    opt_distance_to_surface.append(get_average_distance_to_target_surface
                                   
                                   
                                   (visualization_linkage))
    opt_distance_to_joint.append(get_average_distance_to_target_joint(visualization_linkage))

In [ ]:
dps_iteration_count = len(dps_rest_length)

In [ ]:
print(dps_iteration_count)

In [ ]:
dp_objective_elastic = []
dp_objective_smooth = []
dp_objective_length = []

for iter_idx in range(len(iterateData)):
    dp_objective_elastic.append(iterateData[iter_idx][-4])
    dp_objective_smooth.append(iterateData[iter_idx][-3])
    dp_objective_length.append(iterateData[iter_idx][-2])

dp_total_objective =  np.array([dp_objective_elastic, dp_objective_smooth, dp_objective_length]).sum(axis=0)

In [ ]:
# library
import numpy as np
import matplotlib.pyplot as plt
 
fig, host = plt.subplots()
cmap = plt.get_cmap("Set2")
colors = [cmap(3), cmap(2), cmap(1)]
x=range(len(dp_objective_elastic))
y=np.array([dp_objective_smooth, dp_objective_length, dp_objective_elastic])
 
# Basic stacked area chart.
plt.stackplot(x,y, labels=['Unsmoothness','Length','Elastic'], colors = colors)
plt.legend(loc='upper left')
#plt.show()
# host.set_yscale('log')
fig.set_size_inches(15, 10)

In [ ]:
opt_objective_elastic = []
opt_objective_target = []
opt_objective_smooth = []
opt_objective_length = []

for iter_idx in range(len(opt_iterateData)):
    opt_objective_elastic.append(opt_iterateData[iter_idx][-5])
    opt_objective_target.append(opt_iterateData[iter_idx][-4])
    opt_objective_smooth.append(opt_iterateData[iter_idx][-2])
    opt_objective_length.append(opt_iterateData[iter_idx][-3])
    
opt_objective_elastic = np.array(opt_objective_elastic)
opt_objective_target = np.array(opt_objective_target)
opt_objective_smooth = np.array(opt_objective_smooth)
opt_objective_length = np.array(opt_objective_length)
total_objective =  np.array([opt_objective_elastic, opt_objective_target, opt_objective_smooth, opt_objective_length]).sum(axis=0)

In [ ]:
print(total_objective)

In [ ]:
# library
import numpy as np
import matplotlib.pyplot as plt
 
fig, host = plt.subplots()
x=range(len(opt_objective_elastic))
y=np.array([opt_objective_target, opt_objective_smooth, opt_objective_length, opt_objective_elastic])
cmap = plt.get_cmap("Set2")
colors = [cmap(1), cmap(5), cmap(3), cmap(2)]
# Basic stacked area chart.
plt.stackplot(x,y, labels=['A','B','C', 'D'], colors = colors)
plt.legend(loc='upper left')
#plt.show()
# host.set_yscale('exp')
print(max(total_objective), min(total_objective))
fig.set_size_inches(15, 10)
print(x)
print(y)

### Figure 3E Top Left

In [ ]:
cmap = plt.get_cmap("Set2")
elastic_color = '#555358'
target_color = cmap(1)
rest_length_color = cmap(2)
smoothness_color = cmap(3)
curvature_color = cmap(4)

elastic_label = 'Elastic Energy'
target_label = 'Distance to Surface'
rest_length_label = 'Rest Length Sum'
smoothness_label = 'Smoothing Cost'
curvature_label = 'Curvature Sum'
x_label = 'Iteration'
figure_size = (15, 6)
figure_label_size = 30
design_optimization_iteration_range = [x + dps_iteration_count for x in list(range(len(total_objective)))]

In [ ]:
# library
import numpy as np
import matplotlib.pyplot as plt

fig, host = plt.subplots()
cmap = plt.get_cmap("Set2")
colors = [cmap(3), cmap(2), cmap(1)]
x=range(len(dp_objective_elastic))
y=dp_total_objective
 
# Basic stacked area chart.
plt.plot(x,y, label = 'Objective', color = 'k', linewidth = 3)
#plt.show()
# host.set_yscale('log')
# plt.xlabel('iteration', fontsize = figure_label_size)
plt.ylabel('objective value', fontsize = figure_label_size)
plt.title('Fixed Crossing Initialization', fontsize = figure_label_size)
fig.set_size_inches(figure_size)
fig.savefig('nature_figure3_D_top_left.png', bbox_inches='tight', dpi=400)

### Figure 3E Top Right

In [ ]:
# library
import numpy as np
import matplotlib.pyplot as plt
 
fig, host = plt.subplots()
cmap = plt.get_cmap("Set2")
colors = [cmap(3), cmap(2), cmap(1)]
x=design_optimization_iteration_range
y=total_objective
 
# Basic stacked area chart.
plt.plot(x,y, label = 'Objective', color = 'k', linewidth = 3)
#plt.show()
# host.set_yscale('log')
# plt.xlabel(x_label, fontsize = figure_label_size)
# plt.ylabel('objective value', fontsize = figure_label_size)
plt.title('Design Optimization', fontsize = figure_label_size)

fig.set_size_inches(figure_size)
fig.savefig('nature_figure3_D_top_right.png', bbox_inches='tight', dpi=400)

In [ ]:
combined_energy = (np.array(dps_energy + opt_energy))
combined_rest_length = np.array(dps_rest_length + opt_rest_length)
combined_smoothness = np.array(dps_smoothing + opt_smoothing)
combined_curvature = np.array(dps_total_absolute_curvature + opt_total_absolute_curvature)
combined_dis_to_target_surface = np.array(dps_distance_to_surface + opt_distance_to_surface)
combined_dis_to_target_joint = np.array(dps_distance_to_joint + opt_distance_to_joint)
combined_time_stamps = np.array(dps_time_stamps + opt_time_stamps)

In [ ]:
sum(dps_time_stamps)

In [ ]:
sum(opt_time_stamps[1:])

#### Load data

In [ ]:
# if not os.path.exists("nature_figure3/nature_figure3_plot_data"):
#     os.makedirs("nature_figure3/nature_figure3_plot_data")
# np.save("nature_figure3/nature_figure3_plot_data/{}.npy".format('energy'), combined_energy)
# np.save("nature_figure3/nature_figure3_plot_data/{}.npy".format('rest_length'), combined_rest_length)
# np.save("nature_figure3/nature_figure3_plot_data/{}.npy".format('smoothness'), combined_smoothness)
# np.save("nature_figure3/nature_figure3_plot_data/{}.npy".format('curvature'), combined_curvature)
# np.save("nature_figure3/nature_figure3_plot_data/{}.npy".format('dis_surface'), combined_dis_to_target_surface)
# np.save("nature_figure3/nature_figure3_plot_data/{}.npy".format('dis_joint'), combined_dis_to_target_joint)
# np.save("nature_figure3/nature_figure3_plot_data/{}.npy".format('time_stamps'), combined_time_stamps)
# np.save("nature_figure3/nature_figure3_plot_data/{}.npy".format('dps_total_objective'), dp_total_objective)
# np.save("nature_figure3/nature_figure3_plot_data/{}.npy".format('opt_total_objective'), total_objective)
# np.save("nature_figure3/nature_figure3_plot_data/{}.npy".format('opt_objective_elastic'), opt_objective_elastic)
# np.save("nature_figure3/nature_figure3_plot_data/{}.npy".format('opt_objective_target'), opt_objective_target)
# np.save("nature_figure3/nature_figure3_plot_data/{}.npy".format('opt_objective_smooth'), opt_objective_smooth)
# np.save("nature_figure3/nature_figure3_plot_data/{}.npy".format('opt_objective_length'), opt_objective_length)
# np.save("nature_figure3/nature_figure3_plot_data/{}.npy".format('dp_objective_elastic'), dp_objective_elastic)
# np.save("nature_figure3/nature_figure3_plot_data/{}.npy".format('dp_objective_smooth'), dp_objective_smooth)
# np.save("nature_figure3/nature_figure3_plot_data/{}.npy".format('dp_objective_length'), dp_objective_length)

In [ ]:
# load data
combined_energy = np.load("nature_figure3/nature_figure3_plot_data/{}.npy".format("energy"))
combined_rest_length = np.load("nature_figure3/nature_figure3_plot_data/{}.npy".format("rest_length"))
combined_smoothness = np.load("nature_figure3/nature_figure3_plot_data/{}.npy".format("smoothness"))
combined_curvature = np.load("nature_figure3/nature_figure3_plot_data/{}.npy".format("curvature"))
combined_dis_to_target_surface = np.load("nature_figure3/nature_figure3_plot_data/{}.npy".format('dis_surface'))

dp_total_objective = np.load("nature_figure3/nature_figure3_plot_data/{}.npy".format('dps_total_objective'))
total_objective = np.load("nature_figure3/nature_figure3_plot_data/{}.npy".format('opt_total_objective'))
opt_objective_elastic = np.load("nature_figure3/nature_figure3_plot_data/{}.npy".format('opt_objective_elastic'))
opt_objective_target = np.load("nature_figure3/nature_figure3_plot_data/{}.npy".format('opt_objective_target'))
opt_objective_smooth = np.load("nature_figure3/nature_figure3_plot_data/{}.npy".format('opt_objective_smooth'))
opt_objective_length = np.load("nature_figure3/nature_figure3_plot_data/{}.npy".format('opt_objective_length'))
dp_objective_elastic = np.load("nature_figure3/nature_figure3_plot_data/{}.npy".format('dp_objective_elastic'))
dp_objective_smooth = np.load("nature_figure3/nature_figure3_plot_data/{}.npy".format('dp_objective_smooth'))
dp_objective_length = np.load("nature_figure3/nature_figure3_plot_data/{}.npy".format('dp_objective_length'))

dps_iteration_count = 132

In [ ]:
print(combined_energy[117], combined_rest_length[117], combined_curvature[117])

In [ ]:
print(combined_energy[118], combined_rest_length[118], combined_curvature[118])

In [ ]:
print(combined_energy[119], combined_rest_length[119], combined_curvature[119])

In [ ]:
dps_iteration_count = len(dps_time_stamps)

In [ ]:
print(dps_iteration_count)

### Figure 3E Bottom Left

In [ ]:
figure_label_size = 25
def make_patch_spines_invisible(ax):
    ax.set_frame_on(True)
    ax.patch.set_visible(False)
    for sp in ax.spines.values():
        sp.set_visible(False)


fig, host = plt.subplots()
fig.subplots_adjust(right=0.75)

cmap = plt.get_cmap("Set2")

# plt.axvspan(dps_iteration_count, 144, facecolor=cmap(7), alpha=0.2)

plt.axvline(x=0, color=cmap(6), linestyle='--', label = 'i')
plt.axvline(x=80, color=cmap(6), linestyle='--', label = 'ii')
plt.axvline(x=88, color=cmap(6), linestyle='--', label = 'iii')
plt.axvline(x=105, color=cmap(6), linestyle='--', label = 'iv')
plt.axvline(x=130, color=cmap(6), linestyle='--', label = 'v')



par1 = host.twinx()
par2 = host.twinx()
par3 = host.twinx()
par4 = host.twinx()
# par5 = host.twinx()

# # Offset the right spine of par2.  The ticks and label have already been
# # placed on the right by twinx above.
# par2.spines["right"].set_position(("axes", 1.1))
# # Having been created by twinx, par2 has its frame off, so the line of its
# # detached spine is invisible.  First, activate the frame but make the patch
# # and spines invisible.
# make_patch_spines_invisible(par2)
# # Second, show the right spine. 
# par2.spines["right"].set_visible(False)



par2.spines["left"].set_position(("axes", -0.12))

make_patch_spines_invisible(par2)

par2.spines["left"].set_visible(True)
par2.yaxis.set_label_position('left')
par2.yaxis.set_ticks_position('left')

par3.spines["left"].set_position(("axes", -0.24))

make_patch_spines_invisible(par3)

par3.spines["left"].set_visible(True)
par3.yaxis.set_label_position('left')
par3.yaxis.set_ticks_position('left')



# par5.spines["right"].set_position(("axes", 1.4))
# make_patch_spines_invisible(par5)
# # Second, show the right spine.
# par5.spines["right"].set_visible(True)


p5, = par4.plot(range(len(dps_smoothing)), dps_distance_to_surface, linewidth = 3, color = target_color, label=target_label)
# p6, = par5.plot(range(len(dps_smoothness)), dps_dis_to_target_joint, linewidth = 3, color = cmap(6), label="Weave distance to target joint")

p1, = host.plot(range(len(dps_smoothing)), dps_energy, linewidth = 3, color = elastic_color, label=elastic_label)
p2, = par1.plot(range(len(dps_smoothing)), dps_rest_length, linewidth = 3, color = rest_length_color, label=rest_length_label)
p3, = par2.plot(range(len(dps_smoothing)), dps_smoothing, linewidth = 3, color = smoothness_color, label=smoothness_label)
p4, = par3.plot(range(len(dps_total_absolute_curvature)), dps_total_absolute_curvature, linewidth = 3, color = curvature_color, label=curvature_label, linestyle = '-.')

# p5, = par4.plot(range(dps_iteration_count, dps_iteration_count + len(opt_distance_to_surface)), opt_distance_to_surface, linewidth = 3, color = cmap(5), label="Weave distance to target surface")
# p6, = par5.plot(range(dps_iteration_count, dps_iteration_count + len(opt_distance_to_joint)), opt_distance_to_joint, linewidth = 3, color = cmap(6), label="Weave distance to target joint")


host.set_ylim(min(dps_energy)* 0.8, max(dps_energy)* 1.1)
par1.set_ylim(min(dps_rest_length)* 0.99, max(dps_rest_length)* 1.01)
par2.set_ylim(min(dps_smoothing)* 0.8, max(dps_smoothing)* 1.1)
par3.set_ylim(min(dps_total_absolute_curvature)* 0.8, max(dps_total_absolute_curvature)* 1.01)
par4.set_ylim(min(dps_distance_to_surface)* 0.8, max(dps_distance_to_surface)* 1.01)
# par5.set_ylim(min(dps_dis_to_target_joint)* 0.8, max(dps_dis_to_target_joint)* 1.01)


host.set_yscale('log')


host.set_xlabel(x_label, fontsize = figure_label_size)
host.set_ylabel(elastic_label, fontsize = figure_label_size)
par1.set_ylabel(rest_length_label, fontsize = figure_label_size)
par2.set_ylabel(smoothness_label, fontsize = figure_label_size)
par3.set_ylabel(curvature_label, fontsize = figure_label_size)
par4.set_ylabel(target_label, fontsize = figure_label_size)
# par5.set_ylabel("Weave distance to target joint")

host.yaxis.label.set_color(p1.get_color())
par1.yaxis.label.set_color(p2.get_color())
par2.yaxis.label.set_color(p3.get_color())
par3.yaxis.label.set_color(p4.get_color())
par4.yaxis.label.set_color(p5.get_color())
# par5.yaxis.label.set_color(p6.get_color())

host.yaxis.label.set_fontweight('bold')
par1.yaxis.label.set_fontweight('bold')
par2.yaxis.label.set_fontweight('bold')
par3.yaxis.label.set_fontweight('bold')
par4.yaxis.label.set_fontweight('bold')
# par5.yaxis.label.set_fontweight('bold')

tkw = dict(size=4, width=1.5)
host.tick_params(axis='y', colors=p1.get_color(), **tkw)
par1.tick_params(axis='y', colors=p2.get_color(), **tkw)
par2.tick_params(axis='y', colors=p3.get_color(), **tkw)
par3.tick_params(axis='y', colors=p4.get_color(), **tkw)
par4.tick_params(axis='y', colors=p5.get_color(), **tkw)
# par5.tick_params(axis='y', colors=p6.get_color(), **tkw)
host.tick_params(axis='x', **tkw)

# lines = [p1, p2, p3, p4, p5, p6]
lines = [p1, p2, p3, p4, p5]

par1.yaxis.set_visible(False)
par2.yaxis.set_visible(True)
par3.yaxis.set_visible(True)
par4.yaxis.set_visible(False)
# plt.axvline(x=dps_iteration_count, color=cmap(8), linestyle='--')
# host.legend(lines, [l.get_label() for l in lines], loc="lower left", facecolor='white', framealpha=1, fancybox=True)

fig.set_size_inches(figure_size)
fig.savefig('nature_figure3_D_bottom_left.png', bbox_inches='tight', dpi=400)
plt.show()

### Figure 3E Bottom Right

In [ ]:
figure_label_size = 25
def make_patch_spines_invisible(ax):
    ax.set_frame_on(True)
    ax.patch.set_visible(False)
    for sp in ax.spines.values():
        sp.set_visible(False)


fig, host = plt.subplots()
fig.subplots_adjust(right=0.75)

cmap = plt.get_cmap("Set2")

# plt.axvspan(dps_iteration_count, 144, facecolor=cmap(7), alpha=0.2)

# plt.axvline(x=0, color=cmap(6), linestyle='--', label = 'i')
# plt.axvline(x=80, color=cmap(6), linestyle='--', label = 'ii')
# plt.axvline(x=88, color=cmap(6), linestyle='--', label = 'iii')
# plt.axvline(x=105, color=cmap(6), linestyle='--', label = 'iv')
# plt.axvline(x=130, color=cmap(6), linestyle='--', label = 'v')
plt.axvline(x=4 + dps_iteration_count-1, color=cmap(6), linestyle='--', label = 'vi')


par1 = host.twinx()
par2 = host.twinx()
par3 = host.twinx()
par4 = host.twinx()
# par5 = host.twinx()

# Offset the right spine of par2.  The ticks and label have already been
# placed on the right by twinx above.
par2.spines["right"].set_position(("axes", 1.12))
# Having been created by twinx, par2 has its frame off, so the line of its
# detached spine is invisible.  First, activate the frame but make the patch
# and spines invisible.
make_patch_spines_invisible(par2)
# Second, show the right spine. 
par2.spines["right"].set_visible(False)


par3.spines["right"].set_position(("axes", 1.24))
make_patch_spines_invisible(par3)
# Second, show the right spine.
par3.spines["right"].set_visible(False)

par4.spines["right"].set_position(("axes", 1.12))
make_patch_spines_invisible(par4)
# Second, show the right spine.
par4.spines["right"].set_visible(True)

# par5.spines["right"].set_position(("axes", 1.8))
# make_patch_spines_invisible(par5)
# # Second, show the right spine.
# par5.spines["right"].set_visible(True)

p5, = par4.plot([dps_iteration_count-1] + design_optimization_iteration_range, [dps_distance_to_surface[-1]] + opt_distance_to_surface, linewidth = 3, color = target_color, label=target_label)
# p6, = par5.plot(range(len(opt_smoothness)), opt_dis_to_target_joint, linewidth = 3, color = cmap(6), label="Weave distance to target joint")

p1, = host.plot([dps_iteration_count-1] + design_optimization_iteration_range, [dps_energy[-1]] + opt_energy, linewidth = 3, color = elastic_color, label=elastic_label)
p2, = par1.plot([dps_iteration_count-1] + design_optimization_iteration_range, [dps_rest_length[-1]] + opt_rest_length, linewidth = 3, color = rest_length_color, label=rest_length_label)
p3, = par2.plot([dps_iteration_count-1] + design_optimization_iteration_range, [dps_smoothing[-1]] + opt_smoothing, linewidth = 3, color = smoothness_color, label=smoothness_label)
p4, = par3.plot([dps_iteration_count-1] + design_optimization_iteration_range, [dps_total_absolute_curvature[-1]] + opt_total_absolute_curvature, linewidth = 3, color = curvature_color, label=curvature_label, linestyle = '-.')

# p5, = par4.plot(range(opt_iteration_count, opt_iteration_count + len(opt_distance_to_surface)), opt_distance_to_surface, linewidth = 3, color = cmap(5), label="Weave distance to target surface")
# p6, = par5.plot(range(opt_iteration_count, opt_iteration_count + len(opt_distance_to_joint)), opt_distance_to_joint, linewidth = 3, color = cmap(6), label="Weave distance to target joint")


# host.set_ylim(min(opt_energy)* 0.8, max(opt_energy)* 1.1)
# par1.set_ylim(min(opt_rest_length)* 0.99, max(opt_rest_length)* 1.01)
# par2.set_ylim(min(opt_smoothing)* 0.8, max(opt_smoothing)* 1.1)
# par3.set_ylim(min(opt_total_absolute_curvature)* 0.8, max(opt_total_absolute_curvature)* 1.01)
# par4.set_ylim(min(opt_distance_to_surface)* 0.8, max(opt_distance_to_surface)* 1.01)
# # par5.set_ylim(min(opt_dis_to_target_joint)* 0.8, max(opt_dis_to_target_joint)* 1.01)
host.set_ylim(min(dps_energy)* 0.8, max(dps_energy)* 1.1)
par1.set_ylim(min(dps_rest_length)* 0.99, max(dps_rest_length)* 1.01)
par2.set_ylim(min(dps_smoothing)* 0.8, max(dps_smoothing)* 1.1)
par3.set_ylim(min(dps_total_absolute_curvature)* 0.8, max(dps_total_absolute_curvature)* 1.01)
par4.set_ylim(min(dps_distance_to_surface)* 0.8, max(dps_distance_to_surface)* 1.01)
# par5.set_ylim(min(dps_dis_to_target_joint)* 0.8, max(dps_dis_to_target_joint)* 1.01)




host.set_yscale('log')


host.set_xlabel(x_label, fontsize = figure_label_size)
host.set_ylabel(elastic_label, fontsize = figure_label_size)
par1.set_ylabel(rest_length_label, fontsize = figure_label_size)
par2.set_ylabel(smoothness_label, fontsize = figure_label_size)
par3.set_ylabel(curvature_label, fontsize = figure_label_size)
par4.set_ylabel(target_label, fontsize = figure_label_size)
# par5.set_ylabel("Weave distance to target joint")

host.yaxis.label.set_color(p1.get_color())
par1.yaxis.label.set_color(p2.get_color())
par2.yaxis.label.set_color(p3.get_color())
par3.yaxis.label.set_color(p4.get_color())
par4.yaxis.label.set_color(p5.get_color())
# par5.yaxis.label.set_color(p6.get_color())

host.yaxis.label.set_fontweight('bold')
par1.yaxis.label.set_fontweight('bold')
par2.yaxis.label.set_fontweight('bold')
par3.yaxis.label.set_fontweight('bold')
par4.yaxis.label.set_fontweight('bold')
# par5.yaxis.label.set_fontweight('bold')

tkw = dict(size=4, width=1.5)
# host.tick_params(axis='y', colors=p1.get_color(), **tkw)
par1.tick_params(axis='y', colors=p2.get_color(), **tkw)
par2.tick_params(axis='y', colors=p3.get_color(), **tkw)
par3.tick_params(axis='y', colors=p4.get_color(), **tkw)
par4.tick_params(axis='y', colors=p5.get_color(), **tkw)
# par5.tick_params(axis='y', colors=p6.get_color(), **tkw)
host.tick_params(axis='x', **tkw)

# lines = [p1, p2, p3, p4, p5, p6]
lines = [p1, p2, p3, p4, p5]

host.yaxis.set_visible(False)
par1.yaxis.set_visible(True)
par2.yaxis.set_visible(False)
par3.yaxis.set_visible(False)
par4.yaxis.set_visible(True)
# plt.axvline(x=opt_iteration_count, color=cmap(8), linestyle='--')
host.legend(lines, [l.get_label() for l in lines], loc="upper right", facecolor='white', framealpha=1, fancybox=True, prop={'size': 15})

fig.set_size_inches(figure_size)
fig.savefig('nature_figure3_D_bottom_right.png', bbox_inches='tight', dpi=400)
plt.show()

In [ ]:
def make_patch_spines_invisible(ax):
    ax.set_frame_on(True)
    ax.patch.set_visible(False)
    for sp in ax.spines.values():
        sp.set_visible(False)


fig, host = plt.subplots()
fig.subplots_adjust(right=0.75)

cmap = plt.get_cmap("Set2")

plt.axvspan(dps_iteration_count, 144, facecolor=cmap(7), alpha=0.2)

plt.axvline(x=0, color=cmap(6), linestyle='--', label = 'i')
plt.axvline(x=80, color=cmap(6), linestyle='--', label = 'ii')
plt.axvline(x=88, color=cmap(6), linestyle='--', label = 'iii')
plt.axvline(x=105, color=cmap(6), linestyle='--', label = 'iv')
plt.axvline(x=130, color=cmap(6), linestyle='--', label = 'v')
plt.axvline(x=144, color=cmap(6), linestyle='--', label = 'vi')


par1 = host.twinx()
par2 = host.twinx()
par3 = host.twinx()
par4 = host.twinx()
# par5 = host.twinx()

# Offset the right spine of par2.  The ticks and label have already been
# placed on the right by twinx above.
par2.spines["right"].set_position(("axes", 1.1))
# Having been created by twinx, par2 has its frame off, so the line of its
# detached spine is invisible.  First, activate the frame but make the patch
# and spines invisible.
make_patch_spines_invisible(par2)
# Second, show the right spine.
par2.spines["right"].set_visible(True)


par3.spines["right"].set_position(("axes", 1.2))
make_patch_spines_invisible(par3)
# Second, show the right spine.
par3.spines["right"].set_visible(True)

par4.spines["right"].set_position(("axes", 1.3))
make_patch_spines_invisible(par4)
# Second, show the right spine.
par4.spines["right"].set_visible(True)

# par5.spines["right"].set_position(("axes", 1.4))
# make_patch_spines_invisible(par5)
# # Second, show the right spine.
# par5.spines["right"].set_visible(True)


p5, = par4.plot(range(len(combined_smoothness)), combined_dis_to_target_surface, linewidth = 3, color = target_color, label=target_label)
# p6, = par5.plot(range(len(combined_smoothness)), combined_dis_to_target_joint, linewidth = 3, color = cmap(6), label="Weave distance to target joint")

p1, = host.plot(range(len(combined_smoothness)), combined_energy, linewidth = 3, color = elastic_color, label=elastic_label)
p2, = par1.plot(range(len(combined_smoothness)), combined_rest_length, linewidth = 3, color = rest_length_color, label=rest_length_label)
p3, = par2.plot(range(len(combined_smoothness)), combined_smoothness, linewidth = 3, color = smoothness_color, label=smoothness_label)
p4, = par3.plot(range(len(combined_curvature)), combined_curvature, linewidth = 3, color = curvature_color, label=curvature_label, linestyle = '-.')

# p5, = par4.plot(range(dps_iteration_count, dps_iteration_count + len(opt_distance_to_surface)), opt_distance_to_surface, linewidth = 3, color = cmap(5), label="Weave distance to target surface")
# p6, = par5.plot(range(dps_iteration_count, dps_iteration_count + len(opt_distance_to_joint)), opt_distance_to_joint, linewidth = 3, color = cmap(6), label="Weave distance to target joint")


host.set_ylim(min(combined_energy)* 0.8, max(combined_energy)* 1.1)
par1.set_ylim(min(combined_rest_length)* 0.99, max(combined_rest_length)* 1.01)
par2.set_ylim(min(combined_smoothness)* 0.8, max(combined_smoothness)* 1.1)
par3.set_ylim(min(combined_curvature)* 0.8, max(combined_curvature)* 1.01)
par4.set_ylim(min(combined_dis_to_target_surface)* 0.8, max(combined_dis_to_target_surface)* 1.01)
# par5.set_ylim(min(combined_dis_to_target_joint)* 0.8, max(combined_dis_to_target_joint)* 1.01)


host.set_yscale('log')


host.set_xlabel(x_label)
host.set_ylabel(elastic_label)
par1.set_ylabel(rest_length_label)
par2.set_ylabel(smoothness_label)
par3.set_ylabel(curvature_label)
par4.set_ylabel(target_label)
# par5.set_ylabel("Weave distance to target joint")

host.yaxis.label.set_color(p1.get_color())
par1.yaxis.label.set_color(p2.get_color())
par2.yaxis.label.set_color(p3.get_color())
par3.yaxis.label.set_color(p4.get_color())
par4.yaxis.label.set_color(p5.get_color())
# par5.yaxis.label.set_color(p6.get_color())

host.yaxis.label.set_fontweight('bold')
par1.yaxis.label.set_fontweight('bold')
par2.yaxis.label.set_fontweight('bold')
par3.yaxis.label.set_fontweight('bold')
par4.yaxis.label.set_fontweight('bold')
# par5.yaxis.label.set_fontweight('bold')

tkw = dict(size=4, width=1.5)
host.tick_params(axis='y', colors=p1.get_color(), **tkw)
par1.tick_params(axis='y', colors=p2.get_color(), **tkw)
par2.tick_params(axis='y', colors=p3.get_color(), **tkw)
par3.tick_params(axis='y', colors=p4.get_color(), **tkw)
par4.tick_params(axis='y', colors=p5.get_color(), **tkw)
# par5.tick_params(axis='y', colors=p6.get_color(), **tkw)
host.tick_params(axis='x', **tkw)

# lines = [p1, p2, p3, p4, p5, p6]
lines = [p1, p2, p3, p4, p5]


plt.axvline(x=dps_iteration_count, color=cmap(8), linestyle='--')
host.legend(lines, [l.get_label() for l in lines], loc="lower left", facecolor='white', framealpha=1, fancybox=True)

fig.set_size_inches(figure_size)

fig.savefig('nature_figure3_D_right_all.png', bbox_inches='tight', dpi=400)
plt.show()

#### Visualization

In [ ]:
def make_patch_spines_invisible(ax):
    ax.set_frame_on(True)
    ax.patch.set_visible(False)
    for sp in ax.spines.values():
        sp.set_visible(False)


fig, host = plt.subplots()
fig.subplots_adjust(right=0.75)

cmap = plt.get_cmap("Set2")

plt.axvspan(dps_iteration_count, 144, facecolor=cmap(7), alpha=0.2)

plt.axvline(x=0, color=cmap(6), linestyle='--', label = 'i')
plt.axvline(x=80, color=cmap(6), linestyle='--', label = 'ii')
plt.axvline(x=88, color=cmap(6), linestyle='--', label = 'iii')
plt.axvline(x=105, color=cmap(6), linestyle='--', label = 'iv')
plt.axvline(x=130, color=cmap(6), linestyle='--', label = 'v')
plt.axvline(x=144, color=cmap(6), linestyle='--', label = 'vi')


par1 = host.twinx()
par2 = host.twinx()
par3 = host.twinx()
par4 = host.twinx()
# par5 = host.twinx()

# Offset the right spine of par2.  The ticks and label have already been
# placed on the right by twinx above.
par2.spines["right"].set_position(("axes", 1.1))
# Having been created by twinx, par2 has its frame off, so the line of its
# detached spine is invisible.  First, activate the frame but make the patch
# and spines invisible.
make_patch_spines_invisible(par2)
# Second, show the right spine.
par2.spines["right"].set_visible(True)


par3.spines["right"].set_position(("axes", 1.2))
make_patch_spines_invisible(par3)
# Second, show the right spine.
par3.spines["right"].set_visible(True)

par4.spines["right"].set_position(("axes", 1.3))
make_patch_spines_invisible(par4)
# Second, show the right spine.
par4.spines["right"].set_visible(True)

# par5.spines["right"].set_position(("axes", 1.4))
# make_patch_spines_invisible(par5)
# # Second, show the right spine.
# par5.spines["right"].set_visible(True)


p5, = par4.plot(range(len(combined_smoothness)), combined_dis_to_target_surface, linewidth = 3, color = cmap(5), label="Weave distance to target surface")
# p6, = par5.plot(range(len(combined_smoothness)), combined_dis_to_target_joint, linewidth = 3, color = cmap(6), label="Weave distance to target joint")

p1, = host.plot(range(len(combined_smoothness)), combined_energy, linewidth = 3, color = cmap(1), label="Elastic Energy (log)")
p2, = par1.plot(range(len(combined_smoothness)), combined_rest_length, linewidth = 3, color = cmap(2), label="Total Rest Length")
p3, = par2.plot(range(len(combined_smoothness)), combined_smoothness, linewidth = 3, color = cmap(3), label="Ribbon Smoothness")
p4, = par3.plot(range(len(combined_curvature)), combined_curvature, linewidth = 3, color = cmap(4), label="Ribbon Curvature")

# p5, = par4.plot(range(dps_iteration_count, dps_iteration_count + len(opt_distance_to_surface)), opt_distance_to_surface, linewidth = 3, color = cmap(5), label="Weave distance to target surface")
# p6, = par5.plot(range(dps_iteration_count, dps_iteration_count + len(opt_distance_to_joint)), opt_distance_to_joint, linewidth = 3, color = cmap(6), label="Weave distance to target joint")


host.set_ylim(min(combined_energy)* 0.8, max(combined_energy)* 1.1)
par1.set_ylim(min(combined_rest_length)* 0.99, max(combined_rest_length)* 1.01)
par2.set_ylim(min(combined_smoothness)* 0.8, max(combined_smoothness)* 1.1)
par3.set_ylim(min(combined_curvature)* 0.8, max(combined_curvature)* 1.01)
par4.set_ylim(min(combined_dis_to_target_surface)* 0.8, max(combined_dis_to_target_surface)* 1.01)
# par5.set_ylim(min(combined_dis_to_target_joint)* 0.8, max(combined_dis_to_target_joint)* 1.01)


host.set_yscale('log')


host.set_xlabel("Iterations")
host.set_ylabel("Elastic Energy (log)")
par1.set_ylabel("Total Rest Length")
par2.set_ylabel("Ribbon Smoothness")
par3.set_ylabel("Ribbon Curvature")
par4.set_ylabel("Weave distance to target surface")
# par5.set_ylabel("Weave distance to target joint")

host.yaxis.label.set_color(p1.get_color())
par1.yaxis.label.set_color(p2.get_color())
par2.yaxis.label.set_color(p3.get_color())
par3.yaxis.label.set_color(p4.get_color())
par4.yaxis.label.set_color(p5.get_color())
# par5.yaxis.label.set_color(p6.get_color())

host.yaxis.label.set_fontweight('bold')
par1.yaxis.label.set_fontweight('bold')
par2.yaxis.label.set_fontweight('bold')
par3.yaxis.label.set_fontweight('bold')
par4.yaxis.label.set_fontweight('bold')
# par5.yaxis.label.set_fontweight('bold')

tkw = dict(size=4, width=1.5)
host.tick_params(axis='y', colors=p1.get_color(), **tkw)
par1.tick_params(axis='y', colors=p2.get_color(), **tkw)
par2.tick_params(axis='y', colors=p3.get_color(), **tkw)
par3.tick_params(axis='y', colors=p4.get_color(), **tkw)
par4.tick_params(axis='y', colors=p5.get_color(), **tkw)
# par5.tick_params(axis='y', colors=p6.get_color(), **tkw)
host.tick_params(axis='x', **tkw)

# lines = [p1, p2, p3, p4, p5, p6]
lines = [p1, p2, p3, p4, p5]


plt.axvline(x=dps_iteration_count, color=cmap(8), linestyle='--')
host.legend(lines, [l.get_label() for l in lines], loc="lower left", facecolor='white', framealpha=1, fancybox=True)

fig.set_size_inches(15, 10)
fig.savefig('nature_figure3_D_right_all.png', bbox_inches='tight', dpi=400)
plt.show()

In [ ]:
import plotly.graph_objects as go


x=['Winter', 'Spring', 'Summer', 'Fall']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x, y=[40, 60, 40, 10],
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5, color='rgb(131, 90, 241)'),
    stackgroup='one' # define stack group
))
fig.add_trace(go.Scatter(
    x=x, y=[20, 10, 10, 60],
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5, color='rgb(111, 231, 219)'),
    stackgroup='one'
))
fig.add_trace(go.Scatter(
    x=x, y=[40, 30, 50, 30],
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5, color='rgb(184, 247, 212)'),
    stackgroup='one'
))

fig.update_layout(yaxis_range=(0, 100))
fig.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig = plt.figure()
gs = gridspec.GridSpec(2, 2)

X, Y = np.meshgrid(np.linspace(0,100, num=101), np.linspace(0,100, num=101))
f = lambda x, y: np.sin(x/10.)+y/100.


axMain = plt.subplot(gs[0])
plt.sca(axMain)
cf = axMain.contourf(X[0:40,:], Y[0:40,:], f(X,Y)[0:40,:] , vmin=-1, vmax=2)

divider = make_axes_locatable(axMain)
axShallow = divider.append_axes("top", size="100%", pad=0.1, sharex=axMain)
axShallow.contourf(X[41:80,:], Y[41:80,:], f(X,Y)[41:80,:], vmin=-1, vmax=2)
axShallow.set_xticklabels([])
axShallow2 = divider.append_axes("top", size="50%", pad=0.1, sharex=axMain)
axShallow2.contourf(X[81:,:], Y[81:,:], f(X,Y)[81:,:], vmin=-1, vmax=2)
axShallow2.set_xticklabels([])

plt.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(range(len(combined_smoothness))),
    y=combined_energy,
    name="yaxis1 data"
))


fig.add_trace(go.Scatter(
    x=list(range(len(combined_smoothness))),
    y=combined_rest_length,
    name="yaxis2 data",
    yaxis="y2"
))

fig.add_trace(go.Scatter(
    x=list(range(len(combined_smoothness))),
    y=combined_smoothness,
    name="yaxis3 data",
    yaxis="y3"
))

fig.add_trace(go.Scatter(
    x=list(range(len(combined_smoothness))),
    y=combined_curvature,
    name="yaxis4 data",
    yaxis="y4"
))


# Create axis objects
fig.update_layout(
    xaxis=dict(
        domain=[0.3, 0.7]
    ),
    yaxis=dict(
        title="yaxis title",
        titlefont=dict(
            color="#1f77b4"
        ),
        tickfont=dict(
            color="#1f77b4"
        )
    ),
    yaxis2=dict(
        title="yaxis2 title",
        titlefont=dict(
            color="#ff7f0e"
        ),
        tickfont=dict(
            color="#ff7f0e"
        ),
        anchor="free",
        overlaying="y",
        side="left",
        position=0.15
    ),
    yaxis3=dict(
        title="yaxis4 title",
        titlefont=dict(
            color="#d62728"
        ),
        tickfont=dict(
            color="#d62728"
        ),
        anchor="x",
        overlaying="y",
        side="right"
    ),
    yaxis4=dict(
        title="yaxis5 title",
        titlefont=dict(
            color="#9467bd"
        ),
        tickfont=dict(
            color="#9467bd"
        ),
        anchor="free",
        overlaying="y",
        side="right",
        position=0.85
    )
)

# Update layout properties
fig.update_layout(
    title_text="multiple y-axes example",
    width=800,
)

fig.show()